In [1]:
import numpy as np
import pandas as pd
import pickle

In [73]:
df = pd.DataFrame.from_csv('all_games.csv', index_col=None)
df.head()

,fran_id,pts,opp_fran,opp_pts,game_location,date
0,Rockets,120,Hawks,104,A,1976-10-21
1,Celtics,129,Pacers,122,A,1976-10-21
2,Clippers,133,Bucks,112,A,1976-10-21
3,Lakers,97,Knicks,102,A,1976-10-21
4,Bulls,95,Cavaliers,106,A,1976-10-22


In [74]:
a = df['pts']
b = df['opp_pts']
df['A Win'] = df['pts']>df['opp_pts']
df['A Win'] = df['A Win'].astype(int)
df[:20]

,fran_id,pts,opp_fran,opp_pts,game_location,date,A Win
0,Rockets,120,Hawks,104,A,1976-10-21,1
1,Celtics,129,Pacers,122,A,1976-10-21,1
2,Clippers,133,Bucks,112,A,1976-10-21,1
3,Lakers,97,Knicks,102,A,1976-10-21,0
4,Bulls,95,Cavaliers,106,A,1976-10-22,0
5,Nets,104,Warriors,103,A,1976-10-22,1
6,Nuggets,123,Pacers,110,A,1976-10-22,1
7,Pistons,99,Kings,96,A,1976-10-22,1
8,Suns,98,Jazz,111,A,1976-10-22,0
9,Spurs,121,Sixers,118,A,1976-10-22,1


In [75]:
teams = df['fran_id'].unique()
zeros = np.zeros(df.shape[0])
#print teams

for i in teams:
    #home indices and their points
    indices_home = df[(df['fran_id']==i)].index
    points_home = df[(df['fran_id']==i)].pts
    
    
    #away indices and their points
    indices_away = df[(df['opp_fran']==i)].index
    points_away = df[(df['opp_fran']==i)].pts
    
    #print points_home
    
    #print len(list(indices_home))
    
    # stack the two together and create a df for each team
    ind = (indices_home).tolist() + (indices_away).tolist()
    poi = points_home.tolist() + points_away.tolist()
    #print ind
    #print poi
    teamdf = pd.DataFrame()
    
    teamdf['index'] = ind
    teamdf['points'] = poi
    
    teamdf = teamdf.sort_values('index')
    teamdf = teamdf.reset_index(drop=True)
    teamdf['points'] = pd.rolling_mean(teamdf['points'], 5)
    teamdf.columns = ['index', i]
    teamdf.index = teamdf['index']
    teamdf = teamdf.drop(['index'], axis=1)
    #print teamdf[:20]
    df = df.join(teamdf)
    #print df[-20:]


#print indices

C:\Users\Sterling\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()


In [76]:
df = df.fillna(0)
def findFranAverage(row):
    fran = row['fran_id']
    return row[fran]
def findOppAverage(row):
    opp = row['opp_fran']
    return row[opp]
df['Fran_5_PTS'] = df.apply(findFranAverage,axis=1)
df['Opp_5_PTS'] = df.apply(findOppAverage,axis=1)

for i in teams:
    df = df.drop(i, axis=1)
#print df[teams].sum(axis=1)
#print df[:20]

In [86]:
df[-40:]

,fran_id,pts,opp_fran,opp_pts,game_location,date,A Win,Fran_5_PTS,Opp_5_PTS,fran_elo,opp_elo
47490,Bulls,107,Wizards,112,A,2017-03-17,0,103.0,115.6,1471.232204,1593.801799
47491,Trailblazers,113,Hawks,97,A,2017-03-18,1,107.0,104.2,1503.992255,1528.119240
47492,Jazz,86,Bulls,95,A,2017-03-18,0,95.6,97.2,1578.503147,1464.620623
47493,Wizards,93,Hornets,98,A,2017-03-18,0,108.2,98.2,1600.413380,1419.977355
47494,Rockets,109,Nuggets,105,A,2017-03-18,1,109.6,105.6,1605.443447,1501.593658
47495,Bucks,92,Warriors,117,A,2017-03-18,0,96.8,95.0,1495.916813,1735.782651
47496,Cavaliers,78,Clippers,108,A,2017-03-18,0,97.0,99.4,1628.752979,1541.283647
47497,Spurs,96,Grizzlies,104,A,2017-03-18,0,96.4,99.4,1714.475304,1528.544616
47498,Kings,94,Thunder,110,A,2017-03-18,0,110.2,107.0,1418.394585,1587.809010
47499,Mavericks,111,Nets,104,A,2017-03-19,1,95.0,112.8,1485.023983,1260.741792


In [87]:
df.to_csv('contrived_with_elo.csv')


In [78]:
# Get elo dictionary
elodict = pickle.load(open('../elo_dict.p', 'rb'))

# Add columns to dataframe
df['fran_elo'] = np.zeros(len(df))
df['opp_elo'] = np.zeros(len(df))

In [84]:
# Add elo rating for each team for each game

for team in elodict.keys():
    games = df[(df['fran_id']==team) | (df['opp_fran']==team)]
    home_actual_inds = games[(games['fran_id']==team)].index
    away_actual_inds = games[(games['opp_fran']==team)].index

    games.index = range(len(games))
    home_elo_inds = games[(games['fran_id']==team)].index
    away_elo_inds = games[(games['opp_fran']==team)].index

    away_elos = np.array(elodict[team])[away_elo_inds]
    home_elos = np.array(elodict[team])[home_elo_inds]

    df.ix[home_actual_inds,'fran_elo'] = home_elos
    df.ix[away_actual_inds,'opp_elo'] = away_elos
    
    